In [1]:
from huggingface_hub import login

login(token = "") # Replace YOUR_TOKEN with your Hugging Face token

In [58]:
from tokenizers import Tokenizer

tokenizer_1 = Tokenizer.from_file("/kaggle/input/wordpiece_tokenizer/pytorch/default/1/wordpiece_tokenizer.json")
tokenizer_2 = Tokenizer.from_file("/kaggle/input/bpe_tokenizator/pytorch/default/1/bpe_tokenizer.json")

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [6]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [7]:
import torch
from transformers import pipeline
import torch.nn as nn

original_vocab_size, embedding_dim = model.model.embed_tokens.weight.shape

extra_embedding_1 = nn.Embedding(original_vocab_size, embedding_dim)
extra_embedding_2 = nn.Embedding(original_vocab_size, embedding_dim)

nn.init.xavier_uniform_(extra_embedding_1.weight)
nn.init.xavier_uniform_(extra_embedding_2.weight)

extra_embedding_1.weight.requires_grad = True
extra_embedding_2.weight.requires_grad = True

# Add the new embeddings as attributes
model.extra_embedding_1 = extra_embedding_1
model.extra_embedding_2 = extra_embedding_2

In [51]:
class CustomEmbedding(nn.Module):
    def __init__(self, original_embeddings, new_embeddings_1, new_embeddings_2, tokenizer_1, tokenizer_2):
        super().__init__()
        self.original_embeddings = original_embeddings
        self.new_embeddings_1 = new_embeddings_1
        self.new_embeddings_2 = new_embeddings_2
        self.tokenizer_1 = tokenizer_1
        self.tokenizer_2 = tokenizer_2

    def forward(self, input_ids):
        # Раздели `input_ids` на три части по правилам
        input_ids_llama, input_ids_1, input_ids_2 = input_ids

        # Получи эмбеддинги
        original_embeds = self.original_embeddings(input_ids_llama)
        new_embeds_1 = self.new_embeddings_1(input_ids_1)
        new_embeds_2 = self.new_embeddings_2(input_ids_2)

        # Комбинируем
        combined_embeds = original_embeds + new_embeds_1 + new_embeds_2
        return combined_embeds


# Replace original embedding layer with custom one
model.model.embed_tokens = CustomEmbedding(model.model.embed_tokens, extra_embedding_1, extra_embedding_2, tokenizer_1, tokenizer_2)

In [52]:
# Freeze everething, except the new embeddings
for param in model.parameters():
    param.requires_grad = False

for param in model.extra_embedding_1.parameters():
    param.requires_grad = True

for param in model.extra_embedding_2.parameters():
    param.requires_grad = True

In [ ]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

model.embed_tokens.original_embeddings.original_embeddings.original_embeddings.original_embeddings.original_embeddings.weight False
model.embed_tokens.original_embeddings.original_embeddings.original_embeddings.original_embeddings.new_embeddings_1.weight True
model.embed_tokens.original_embeddings.original_embeddings.original_embeddings.original_embeddings.new_embeddings_2.weight True
model.layers.0.self_attn.q_proj.weight False
model.layers.0.self_attn.k_proj.weight False
model.layers.0.self_attn.v_proj.weight False
model.layers.0.self_attn.o_proj.weight False
model.layers.0.mlp.gate_proj.weight False
model.layers.0.mlp.up_proj.weight False
model.layers.0.mlp.down_proj.weight False
model.layers.0.input_layernorm.weight False
model.layers.0.post_attention_layernorm.weight False
model.layers.1.self_attn.q_proj.weight False
model.layers.1.self_attn.k_proj.weight False
model.layers.1.self_attn.v_proj.weight False
model.layers.1.self_attn.o_proj.weight False
model.layers.1.mlp.gate_proj.we

In [56]:
from datasets import load_dataset

ds = load_dataset("mlabonne/FineTome-Alpaca-100k", split="train")


In [12]:
print(ds[0])

{'source': 'infini-instruct-top-500k', 'score': 5.212620735168457, 'instruction': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programm

In [61]:
def tokenize(examples):
    texts = [
        f"### Instruction: {instruction}\n### Response: {output}" 
        for instruction, output in zip(examples['instruction'], examples['output'])
    ]
    
    tokenized_llama = tokenizer(
        texts,
        truncation=True,
        max_length=1024,  
        padding="max_length",
        return_tensors=None  
    )
    tokenized_1 = tokenizer_1.encode_batch(texts)
    tokenized_2 = tokenizer_2.encode_batch(texts)

    input_ids_llama = tokenized_llama["input_ids"].copy()
    input_ids_1 = [t.ids for t in tokenized_1]
    input_ids_2 = [t.ids for t in tokenized_2]

    
    attention_mask = tokenized_llama["attention_mask"]
    
    return {
        "input_ids_llama": input_ids_llama,
        "input_ids_1": input_ids_1,
        "input_ids_2": input_ids_2,
        "attention_mask": attention_mask,
        "labels": input_ids_llama  # Метки берем из основного токенизатора
    }

tokenizer.pad_token = tokenizer.eos_token

tokenized_ds = ds.map(
    tokenize, 
    batched=True,
    remove_columns=['instruction',"source","score",'output']  
)

tokenized_ds.set_format(type="torch", columns=["input_ids_llama", "input_ids_1", "input_ids_2", "attention_mask", "labels"])

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [20]:
import wandb
wandb.login(key="")

wandb: Currently logged in as: nazgulka2005 (nazgulka2005-innopolis-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import TrainingArguments, Trainer, TrainerCallback
import wandb

wandb.init(project="custom_embedding_training")

In [ ]:
from transformers import PreTrainedTokenizerFast

tokenizer_1 = PreTrainedTokenizerFast(tokenizer_file="/kaggle/input/wordpiece_tokenizer/pytorch/default/1/wordpiece_tokenizer.json")
tokenizer_2 = PreTrainedTokenizerFast(tokenizer_file="/kaggle/input/bpe_tokenizator/pytorch/default/1/bpe_tokenizer.json")


tokenizer_1.pad_token = "[PAD]"

tokenizer_1.pad_token_id = tokenizer_1.convert_tokens_to_ids("[PAD]")


In [ ]:
if tokenizer_2.pad_token_id is None:
    tokenizer_2.add_special_tokens({"pad_token": "[PAD]"})
    tokenizer_2.pad_token_id = tokenizer_2.convert_tokens_to_ids("[PAD]")

print("Updated tokenizer_2.pad_token_id:", tokenizer_2.pad_token_id)

Updated tokenizer_2.pad_token_id: 128256


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

optimizer = torch.optim.AdamW([
    {"params": model.extra_embedding_1.parameters(), "lr": 2e-5},
    {"params": model.extra_embedding_2.parameters(), "lr": 2e-5}
])

class CustomDataCollator:
    def __call__(self, batch):
        input_ids_llama = pad_sequence([item["input_ids_llama"] for item in batch], batch_first=True, padding_value=tokenizer.pad_token_id)
        input_ids_1 = pad_sequence([item["input_ids_1"] for item in batch], batch_first=True, padding_value=tokenizer_1.pad_token_id)
        input_ids_2 = pad_sequence([item["input_ids_2"] for item in batch], batch_first=True, padding_value=tokenizer_2.pad_token_id)
        attention_mask = pad_sequence([item["attention_mask"] for item in batch], batch_first=True, padding_value=0)
        labels = pad_sequence([item["labels"] for item in batch], batch_first=True, padding_value=-100)

        return {
            "input_ids_llama": input_ids_llama,
            "input_ids_1": input_ids_1,
            "input_ids_2": input_ids_2,
            "attention_mask": attention_mask,
            "labels": labels
        }

data_collator = CustomDataCollator()

training_args = TrainingArguments(
    evaluation_strategy="no",
    eval_strategy="steps",
    output_dir="./trained_embeddings",
    per_device_train_batch_size=4,  
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    eval_steps=50,
    save_strategy="epoch",
    learning_rate=2e-5,
    remove_unused_columns=False,  
    fp16=True,  
    optim="adamw_torch",
    report_to=["wandb"]
)

class CustomTrainingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            wandb.log(logs)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(optimizer, None),  
    callbacks=[CustomTrainingCallback()]
)

trainer.train()

wandb.finish()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-66-168f6d421c39>:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: Caught ValueError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/parallel_apply.py", line 96, in _worker
    output = module(*input, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/llama/modeling_llama.py", line 1163, in forward
    outputs = self.model(
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/l